# 20240718_HB_conjoint_interactions

    Project: JMR23 - Disentangling Watches
    Author: Alex Burnap
    Email: alex.burnap@yale.edu
    Date: July 18, 2024
    License: MIT



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%load_ext autoreload
%autoreload 2

/tmp/ipykernel_2679027/1752212046.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import os
import math
import glob
# import skimage
import numpy
# import cv2
import pandas as pd
import statsmodels as sm
import statsmodels.formula.api as smf
# from PIL import Image
import matplotlib.pyplot as plt

import pymc as pm
import arviz as az
# import graphviz
# import pytensor as pt
# import aesara.tensor as at

az.rcParams["plot.max_subplots"] = 100

In [3]:
# Bayesian Libs
import pymc as pm
import arviz as az
import jax
import pymc.sampling_jax as sampling_jax

/home/pangaea-gaia/anaconda3/envs/pymc/lib/python3.11/site-packages/pymc/sampling/jax.py:39: UserWarning: This module is experimental.
  warnings.warn("This module is experimental.")


In [4]:
RANDOM_SEED = 0

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = str(RANDOM_SEED)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
from data_interactions import get_data, generate_pairwise_names, generate_pairwise_values
# from run_HB_conjoint import run_HB_conjoint_model

In [6]:
# Globals
NUM_RESPONDENTS = 253
NUM_ALTERNATIVES = 2
NUM_ATTRIBUTES = 6
NUM_ATTRIBUTES_PLUS_INTERACTIONS = 6 + 15 # attributes + pairwise interactions
NUM_TASKS = 15
NUM_TRAIN_TASKS = 13
# NUM_TRAIN_TASKS = 15
NUM_TEST_TASKS = 2
NUM_COVARIATES = 6

np.random.seed(RANDOM_SEED)

In [7]:
COVARIATE_NAMES = ["DemoGender_male",
                    "DemoGender_female",
                    "DemoAge_real",
                    "DemoIncome_real",
                    "DemoEducation_real",
                    "DemoAestheticImportance1_1_real"]

VISUAL_ATTRIBUTE_NAMES =  ["dialcolor",
                         "dialshape",
                         "strapcolor",
                         "dialsize",
                         "knobsize",
                         "rimcolor"]

In [8]:
choice_and_demo_coords_with_interactions = {"visual_attributes": ["dialcolor",
                                                                  "dialshape",
                                                                  "strapcolor",
                                                                  "dialsize",
                                                                  "knobsize",
                                                                  "rimcolor"],
                                            
                                            "visual_attribute_interactions": ['dialcolor_dialshape', 'dialcolor_strapcolor',
                                                                              'dialcolor_dialsize', 'dialcolor_knobsize', 'dialcolor_rimcolor',
                                                                              'dialshape_strapcolor', 'dialshape_dialsize', 'dialshape_knobsize',
                                                                              'dialshape_rimcolor', 'strapcolor_dialsize', 'strapcolor_knobsize',
                                                                              'strapcolor_rimcolor', 'dialsize_knobsize', 'dialsize_rimcolor',
                                                                              'knobsize_rimcolor'],

                                              "covariates": ["DemoGender_male",
                                                             "DemoGender_female",
                                                             "DemoAge_real",
                                                             "DemoIncome_real",
                                                             "DemoEducation_real",
                                                             "DemoAestheticImportance1_1_real",
                                                             # "DemoTimeSpentGrooming_real" # left out
                                                             ],

                                              "resp_ind": range(NUM_RESPONDENTS),
                                              "task_ind": range(NUM_TRAIN_TASKS)}

# Get Data

In [9]:
# Create random Train/Valid/Test Indices
TRAIN_TASK_INDICES = np.sort(np.random.choice(range(NUM_TASKS), NUM_TRAIN_TASKS, replace=False))
TEST_TASK_INDICES = np.setdiff1d(range(NUM_TASKS), TRAIN_TASK_INDICES)

In [10]:
X_train, X_test, Y_train, Y_test, Z_df = get_data(TRAIN_TASK_INDICES, TEST_TASK_INDICES)

In [11]:
X_train.shape

(253, 13, 2, 6)

## Generate pairwise interactions

In [12]:
X_train = generate_pairwise_values(X_train, num_respondents= NUM_RESPONDENTS, num_tasks=NUM_TRAIN_TASKS, num_alternatives=NUM_ALTERNATIVES)
X_train.shape

(253, 13, 2, 21)

In [13]:
X_test = generate_pairwise_values(X_test, num_respondents= NUM_RESPONDENTS, num_tasks=NUM_TEST_TASKS, num_alternatives=NUM_ALTERNATIVES)
X_test.shape

(253, 2, 2, 21)

# Train HB Conjoint with Interactions Model

In [15]:
def run_HB_conjoint_model(X_train,
                          Y_train,
                          Z_df,
                          choice_and_demo_coords=None,
                          use_pairwise_interactions=False,
                          pairwise_interaction_std=None,
                          mu_theta_hyper_std=0.25,
                          theta_std=0.25,
                          beta_covar_dist=0.5,
                          beta_covar_eta=3.0,
                          num_draws=2000,
                          num_tune=2000,
                          num_chains=1, # num of GPUs is most efficient for RAM -> VRAM
                          target_accept=0.65,
                          random_seed=RANDOM_SEED,
                          ):

    print("Using PyMC version: {}. Please ensure ver >= 5 for GPU support with Jax/Numpyro".format(pm.__version__))
    print("Using {} for MCMC sampling".format(jax.default_backend()))
    print("Sampling Devices: {}".format(jax.devices()))
    print("Note: No progress bar if > 1 GPU\n")
    
    if use_pairwise_interactions:
        assert pairwise_interaction_std is not None
        num_attributes = len(choice_and_demo_coords["visual_attributes"])
        X_train, X_train_interactions = X_train[:, :, :, :num_attributes], X_train[:, :, :, num_attributes:]
        print("****************************************\n              Using Pairwise Interactions              \n****************************************")
    else:
        print("****************************************\n              NOT Using Pairwise Interactions              \n****************************************")
    
    assert choice_and_demo_coords is not None
    num_attributes = len(choice_and_demo_coords["visual_attributes"])
    
    

    # Define HB Model - Wrap in self-context
    with pm.Model(coords=choice_and_demo_coords) as HB_ind_seg_model:

        #     X_attributes_left = pm.MutableData("X_attributes_left", X[:,:,0,:], dims=("resp_ind", "task_ind", "visual_attributes"))
        #     X_attributes_right = pm.MutableData("X_attributes_right", X[:,:,1,:], dims=("resp_ind", "task_ind", "visual_attributes"))
        X_attributes_left = pm.ConstantData("X_attributes_left",
                                            X_train[:, :, 0, :],
                                            dims=("resp_ind", "task_ind", "visual_attributes"))

        X_attributes_right = pm.ConstantData("X_attributes_right",
                                             X_train[:, :, 1, :],
                                             dims=("resp_ind", "task_ind", "visual_attributes"))
        
        if use_pairwise_interactions:
            X_interactions_left = pm.ConstantData("X_interactions_left",
                                            X_train_interactions[:, :, 0, :],
                                            dims=("resp_ind", "task_ind", "visual_attribute_interactions"))

            X_interactions_right = pm.ConstantData("X_interactions_right",
                                                 X_train_interactions[:, :, 1, :],
                                                 dims=("resp_ind", "task_ind", "visual_attribute_interactions"))

        covariates = pm.ConstantData("covariate_vars",
                                     Z_df.to_numpy(),
                                     dims=('resp_ind', 'covariates'))

        # Level 3: Hyperprior
        mu_theta_hyper = pm.Normal('beta_mu_hyper',
                                   mu=0,
                                   sigma=mu_theta_hyper_std,
                                   dims=('covariates', 'visual_attributes'))

        #  Level 2: Population Prior
        thetas = pm.Normal('thetas',
                           mu=mu_theta_hyper,
                           sigma=theta_std,
                           dims=('covariates', 'visual_attributes'))

        mu_beta = pm.Deterministic("mu_betas",
                                   pm.math.dot(covariates, thetas),
                                   dims=('resp_ind', 'visual_attributes'))

        # Covariance Matrix - LKJ Cholesky instead of Inverse-Wishart
        chol_beta, corr_beta, stds_beta = pm.LKJCholeskyCov("chol_beta",
                                                            n=num_attributes,
                                                            eta=beta_covar_eta,
                                                            sd_dist=pm.Exponential.dist(beta_covar_dist),
                                                            compute_corr=True)
        cov_beta = pm.Deterministic("cov_beta",
                                    chol_beta.dot(chol_beta.T))

        betas = pm.MvNormal("betas", mu_beta,
                            chol=chol_beta,
                            dims=('resp_ind', 'visual_attributes'))

        beta_mean = pm.Deterministic("beta_mean",
                                     betas.mean(axis=0))
        
        if use_pairwise_interactions:
            beta_interactions = pm.Normal('beta_interactions',
                                           mu=0,
                                           sigma=pairwise_interaction_std,
                                           dims=('visual_attribute_interactions'))

        # Level 1: Choice Likelihood (Bernoulli)
        U_left = (betas[:, None, :] * X_attributes_left).sum(axis=2)
        U_right = (betas[:, None, :] * X_attributes_right).sum(axis=2)
        
        if use_pairwise_interactions:
            U_left_interactions = (beta_interactions * X_interactions_left).sum(axis=2)
            U_right_interactions = (beta_interactions * X_interactions_right).sum(axis=2)
            U_left = U_left + U_left_interactions
            U_right = U_right + U_right_interactions

        p = pm.Deterministic("p", (pm.math.sigmoid(U_right) / (pm.math.sigmoid(U_left) + pm.math.sigmoid(U_right))), dims=("resp_ind", "task_ind"))

        # likelihood
        pm.Bernoulli("y",
                     p=p,
                     observed=Y_train,
                     #                  observed=Y,
                     #                 observed=valid_choices_and_demo_df["Response"],
                     dims=("resp_ind", "task_ind"))

        idata_seg = sampling_jax.sample_numpyro_nuts(draws=num_draws,
                                                     tune=num_tune,
                                                     chains=num_chains,
                                                     target_accept=target_accept,
                                                     random_seed=random_seed)
    return HB_ind_seg_model, idata_seg

In [16]:
HB_ind_seg_model, idata_seg = run_HB_conjoint_model(X_train=X_train,
                                                   Y_train=Y_train,
                                                   Z_df=Z_df,
                                                   choice_and_demo_coords=choice_and_demo_coords_with_interactions,
                                                   use_pairwise_interactions=True,
                                                   pairwise_interaction_std=0.25,
                                                   mu_theta_hyper_std=0.25,
                                                   theta_std=0.25,
                                                   beta_covar_dist=0.5,
                                                   beta_covar_eta=3.0,
                                                   num_draws=2000,
                                                   num_tune=2000,
                                                   num_chains=1, # num of GPUs is most efficient for RAM -> VRAM
                                                   target_accept=0.65,
                                                   random_seed=RANDOM_SEED
                                                   )

Using PyMC version: 5.0.2. Please ensure ver >= 5 for GPU support with Jax/Numpyro
Using gpu for MCMC sampling
Sampling Devices: [StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)]
Note: No progress bar if > 1 GPU

****************************************
              Using Pairwise Interactions              
****************************************


/home/pangaea-gaia/anaconda3/envs/pymc/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Compiling...
Compilation time =  0:00:01.421718
Sampling...


sample: 100%|█| 4000/4000 [09:58<00:00,  6.69it/s, 127 steps of size 4.01e-02. a


Sampling time =  0:10:12.299281
Transforming variables...
Transformation time =  0:00:01.715607


# In- and Out-of-Sample Prediction WITH interaction terms (partially Frequentist)

## In-Sample

In [17]:
mu_beta_means = idata_seg.posterior["betas"].to_numpy().mean(axis=0).mean(axis=0)
mu_beta_interactions = idata_seg.posterior["beta_interactions"].to_numpy().mean(axis=0).mean(axis=0)

In [18]:
# In-Sample - Multiply with left and right conjoint designs
left_utility_train = (np.expand_dims(mu_beta_means, axis=1) * X_train[:,:,0,:6]).sum(axis=2)
right_utility_train = (np.expand_dims(mu_beta_means, axis=1) * X_train[:,:,1,:6]).sum(axis=2)

left_utility_interactions_train = np.dot(X_train[:,:,0,6:], np.expand_dims(mu_beta_interactions, axis=1)).squeeze(2)
right_utility_interactions_train = np.dot( X_train[:,:,1,6:], np.expand_dims(mu_beta_interactions, axis=1)).squeeze(2)

In [20]:
# In-Sample Accuracy
y_hat_binary_train = ((right_utility_train + right_utility_interactions_train) > (left_utility_train + left_utility_interactions_train)) * 1
((Y_train == y_hat_binary_train)*1).mean()

0.8713894800851323

## Out-of-Sample (Final Results)

In [21]:
# Out of Sample - Multiply with left and right conjoint designs
left_utility_test = (np.expand_dims(mu_beta_means, axis=1) * X_test[:,:,0,:6]).sum(axis=2)
right_utility_test = (np.expand_dims(mu_beta_means, axis=1) * X_test[:,:,1,:6]).sum(axis=2)

left_utility_interactions_test = np.dot(X_test[:,:,0,6:], np.expand_dims(mu_beta_interactions, axis=1)).squeeze(2)
right_utility_interactions_test = np.dot( X_test[:,:,1,6:], np.expand_dims(mu_beta_interactions, axis=1)).squeeze(2)

# Out of Sample Accuracy
y_hat_binary_test = (right_utility_test > left_utility_test) * 1
((Y_test == y_hat_binary_test)*1).mean()

0.6877470355731226

In [2]:
# Seed 0, 1, 2 - Results for interactions model
accuracies_HB_conjoint_interactions = np.array([0.6877470355731226, 0.7134387351778656, 0.7233201581027668])
accuracies_HB_conjoint_interactions.mean(), accuracies_HB_conjoint_interactions.std()

(0.7081686429512516, 0.01499315767686567)